# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096795


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:46,  3.80s/it]

Rendering models:  13%|█▎        | 4/30 [00:08<01:13,  2.82s/it]

Rendering models:  17%|█▋        | 5/30 [00:09<00:54,  2.18s/it]

Rendering models:  20%|██        | 6/30 [00:09<00:38,  1.62s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:23<00:01,  1.67it/s]

Rendering models:  93%|█████████▎| 28/30 [00:24<00:01,  1.71it/s]

Rendering models:  97%|█████████▋| 29/30 [00:25<00:00,  1.42it/s]

Rendering models: 100%|██████████| 30/30 [00:25<00:00,  1.72it/s]

Jnursssmith5263                       0.001286
DannyU                                0.000601
jnarayanbvg                           0.009903
izzyathogwarts                        0.019958
deivad                                0.004849
ellivc                                0.000578
not-logged-in-c6917d9b19f41e6c2995    0.001621
not-logged-in-c6917d9b19f41e6c2995    0.001635
awright5                              0.001381
KjellN                                0.008752
not-logged-in-3bf20897ad3067a962a6    0.000524
browny811                             0.000655
Theaker                               0.003089
acapirala                             0.000801
Lavadude                              0.026174
ShirleyL                              0.000513
w7250369                              0.000843
graffiti                              0.000614
lgonzalez62                           0.002532
Tessaje04                             0.072812
ykuzmits                              0.000626
h4ranek      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())